In [21]:
from flask import Flask, render_template, request
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from keras.utils import to_categorical


In [9]:
cols = ['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall']
z_thresh = 3
def remove_outliers(df, cols, z_thresh):
    for col in cols:
        z_scores = np.abs((df[col] - df[col].mean()) / df[col].std())
        df = df[z_scores < z_thresh]
    return df


In [10]:

# Define a function to load and preprocess the data
def load_data():
    # Load the dataset
    data = pd.read_csv('Crop_recommendation.csv')
    data = remove_outliers(data,cols,z_thresh)
    # Preprocess the data
    le = LabelEncoder()
    data['label'] = le.fit_transform(data['label'])
    X = data.drop('label', axis=1)
    y = data['label']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    return X_train, X_test, y_train, y_test



In [24]:
X_train, X_test, y_train, y_test = load_data()
to_categorical(y_train).shape

(1624, 22)

In [27]:
X_train

,N,P,K,temperature,humidity,ph,rainfall
1852,7,21,35,25.760117,94.658306,5.764812,131.245141
934,8,25,36,19.913305,94.950314,6.828522,104.027706
789,60,59,22,31.868473,66.742175,7.191523,74.222386
586,15,54,15,29.976043,57.031844,8.354958,44.860529
2103,108,24,31,24.128325,56.181077,6.431900,147.275782
...,...,...,...,...,...,...,...
1176,14,18,35,31.091542,47.020584,4.791147,91.466643
1397,96,18,50,25.331045,84.305338,6.904242,41.532187
906,21,21,38,22.552606,89.325949,6.327674,104.895564
1605,26,27,10,28.069032,92.914873,6.079998,114.133942


In [25]:
# Define a function to build the model
def build_model():
    # Load the data
    X_train, X_test, y_train, y_test = load_data()
    y_train = to_categorical(y_train, num_classes=22)
    y_test = to_categorical(y_test, num_classes=22)
    
    # Define the model architecture
    model = Sequential()
    model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(22, activation='softmax'))
    # Compile the model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    # Train the model
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=64)
    
    return model

# Train the model
model = build_model()






Epoch 1/100
26/26 [==============================] - 1s 8ms/step - loss: 14.3475 - accuracy: 0.1034 - val_loss: 2.6786 - val_accuracy: 0.2833
Epoch 2/100
26/26 [==============================] - 0s 3ms/step - loss: 4.8092 - accuracy: 0.2180 - val_loss: 1.7437 - val_accuracy: 0.4901
Epoch 3/100
26/26 [==============================] - 0s 4ms/step - loss: 2.8379 - accuracy: 0.2734 - val_loss: 1.7945 - val_accuracy: 0.5542
Epoch 4/100
26/26 [==============================] - 0s 3ms/step - loss: 2.3830 - accuracy: 0.3220 - val_loss: 1.6549 - val_accuracy: 0.5813
Epoch 5/100
26/26 [==============================] - 0s 3ms/step - loss: 2.1820 - accuracy: 0.3830 - val_loss: 1.4642 - val_accuracy: 0.6527
Epoch 6/100
26/26 [==============================] - 0s 4ms/step - loss: 1.8572 - accuracy: 0.4631 - val_loss: 1.1522 - val_accuracy: 0.7241
Epoch 7/100
26/26 [==============================] - 0s 3ms/step - loss: 1.6737 - accuracy: 0.5135 - val_loss: 0.8884 - val_accuracy: 0.7980
Epoch 8/100


In [26]:
model.save('my_model.h5')
